In [8]:
from dotenv import load_dotenv
import os
import requests
import pandas as pd
import json
import datetime
from dataclasses import dataclass,asdict
from time import sleep

In [9]:
CLEF_API = "API_KEY"

In [10]:
load_dotenv()
api_key = os.getenv("API_KEY")

In [11]:
def gare_page(page_num):
    '''Requete une liste de gare à une page spécifique'''
    reponse = requests.get(f"https://api.sncf.com/v1/coverage/sncf/stop_areas?start_page={page_num}", auth=(api_key,""))
    return reponse.json()

In [12]:
def nombre_pages():
    '''Calcul le nombre de pages de la requete'''

    #Je fais une premiere requete
    premiere_page = gare_page(0)
    #Je recupère le nombre de gares total et le nombre de gare par requete
    nb_gares = premiere_page["pagination"]["total_result"]
    nb_gares_par_page = premiere_page["pagination"]["items_per_page"]

    nb_pages = nb_gares // nb_gares_par_page
    # On calcul le modulo (reste de la division)
    modulo = nb_gares % nb_gares_par_page
    # Si le reste de la division est différent de 0, alors on ajoute une dernière page
    if modulo != 0:
        nb_pages += 1
    
    print(f"Il y a {nb_gares_par_page} gares par page, et {nb_gares} gares au total. Il y a donc {nb_pages} au total")
    return nb_pages

In [13]:
def ecrire_json(data,nom_fichier):
    with open(nom_fichier, "w") as fc:
        json.dump(data, fc)

In [14]:
@dataclass
class DataClassGare:
    id : str | None
    nom : str | None
    timezone : str | None
    label : str | None
    longitude : str | None
    latitude : str | None
    id_region_ad : str | None
    nom_region_ad : str | None
    zip_code : str | None
    insee_region_ad : str | None


In [15]:
def extraire_donnees(liste_gares,clef):
    '''Cette fonction permet d'extraire une donnée dans un dictionnaire
    Si la donnée est de profondeur 1 la clef est un string
    Si la donnée est de profondeur > 1 la clef est une liste '''

    try:
        # Extraire une donnée de profondeur 1
        if type(clef) is str:
            data = liste_gares[clef]
        # Extraire une donnée de profondeur > 1
        else:
            data = liste_gares
            # Gestion du cas particulier de la gestion administrative
            if clef[0] == "administrative_regions":
                for x in clef[1:]:
                    data = liste_gares["administrative_regions"][0][x]
            # autres cas
            else:
                for x in clef:
                    data = data[x]
        return data
    
    except KeyError:
        return None

In [16]:
def recuperer_data(donnees):
    '''extrait des données et les retourne sous forme de dictionnaire'''
    # Je recupère toutes les donnéee de toutes les gares.

    gare = DataClassGare(
        id = extraire_donnees(donnees,"id"),
        nom = extraire_donnees(donnees,"name"),
        timezone=extraire_donnees(donnees,"timezone"),
        label=extraire_donnees(donnees,"label"),
        longitude=extraire_donnees(donnees,["coord","lon"]),
        latitude=extraire_donnees(donnees,["coord","lat"]),
        id_region_ad=extraire_donnees(donnees,["administrative_regions","id"]),
        nom_region_ad=extraire_donnees(donnees,["administrative_regions","name"]),
        zip_code=extraire_donnees(donnees,["administrative_regions","zip_code"]),
        insee_region_ad=extraire_donnees(donnees,["administrative_regions","insee"])
    )

    return asdict(gare)

In [17]:
def run(nb_pages):

    #Initialisation de la liste de toutes les gares
    toutes_gares_propres = []

    #J'itère sur chaque page pour faire une requete
    for page in range(nb_pages):
        requete = gare_page(page)
        #Je stocke les données de toutes les gares dans une variable
        gares = requete["stop_areas"]

        if page % 10 == 0:
            print(f"Je suis à la page {page}")

        #J'itère sur chaque gare
        for gare in gares:
            #J'extrait les données
            data_gare = recuperer_data(gare)
            #J'ajoute les données extraites à ma liste finale
            toutes_gares_propres.append(data_gare)
        #Delais de 3s avant prochaine requete
        sleep(3)
    
    return toutes_gares_propres
    
        

In [18]:
def data_to_dataframe(data):
    '''Cette fonction creer un dataframe et place les données dans le dataframe'''

    # Creation du dataframe avec pour nom de colonne la clé de chaque valeur.
    df = pd.DataFrame(columns=data[0].keys())
    # Insert les données ligne par ligne
    for item in data:
        #Le nombre de ligne est l'indice auquel je veut inserer mes données
        nb_lignes = len(df)
        #J'insert mes données dans la derniere ligne
        df.loc[nb_lignes] = item

    return df

In [ ]:
load_dotenv()
api_key = os.getenv("API_KEY")

nb_pages = nombre_pages()
toutes_les_gares = run(nb_pages)
dataframe = data_to_dataframe(toutes_les_gares)
dataframe.to_csv("liste_gares.csv", index=False, sep=",")